<a href="https://colab.research.google.com/github/hakimamerabet/book_recommendation/blob/hakima/book_recommendation_project_preprocessing_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import wordcloud
import en_core_web_sm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
#data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/book_recommendation_project/books_data.csv')

In [ ]:
data = pd.read_csv("books_data.csv")

In [ ]:
data.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [ ]:
# Basic stats
def basic_stats(dataframe):

    print("Display of dataset: ")
    display(dataframe.head())

    print("Basics statistics: ")
    display(dataframe.describe(include="all"))

    print("Percentage of missing values: ")
    display(100 * dataframe.isnull().sum() / dataframe.shape[0])

    print('Type for each columns')
    display(dataframe.dtypes)

    print(f"Dataframe shape : {dataframe.shape[0]} rows, {dataframe.shape[1]} columns")
    print()
    print('Count of duplicates rows :', len(dataframe)-len(dataframe.drop_duplicates()))
    print()

basic_stats(data)

Display of dataset: 


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


Basics statistics: 


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
count,212403,143962,180991,160329,188568,136518,187099,188568,171205,49752.000000
unique,212403,133226,127278,149387,188099,16016,11582,184506,10883,NaN
top,Its Only Art If Its Well Hung!,Unlike some other reproductions of classic tex...,['Rose Arny'],http://books.google.com/books/content?id=7dMSA...,http://books.google.com/books?id=acwPAgAAQBAJ&...,Simon and Schuster,2000,https://play.google.com/store/books/details?id...,['Fiction'],NaN
freq,1,92,236,79,17,3454,4286,18,23419,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.252975
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201.340431
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000


Percentage of missing values: 


,0
Title,0.000471
description,32.222557
authors,14.789270
image,24.516958
previewLink,11.222011
publisher,35.727199
publishedDate,11.913617
infoLink,11.222011
categories,19.396527
ratingsCount,76.576712


Type for each columns


,0
Title,object
description,object
authors,object
image,object
previewLink,object
publisher,object
publishedDate,object
infoLink,object
categories,object
ratingsCount,float64


Dataframe shape : 212404 rows, 10 columns

Count of duplicates rows : 0



In [ ]:
data.rename(columns={'Title': 'title'}, inplace=True)

In [ ]:
data = data.map(lambda x: x.replace('[', '').replace(']', '').replace("'", '') if isinstance(x, str) else x)
data.head(20)

,title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,Julie Strain,http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,Comics & Graphic Novels,NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,Philip Nel,http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,Biography & Autobiography,NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,David R. Ray,http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,Religion,NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,Veronica Haddon,http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,Fiction,NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,Edward Long,NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN
5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,Everett Ferguson,http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,Religion,5.0
6,The Overbury affair (Avon),NaN,Miriam Allen De Ford,NaN,http://books.google.nl/books?id=mHLTngEACAAJ&d...,NaN,1960,http://books.google.nl/books?id=mHLTngEACAAJ&d...,NaN,NaN
7,A Walk in the Woods: a Play in Two Acts,NaN,Lee Blessing,NaN,http://books.google.nl/books?id=6HDOwAEACAAJ&d...,NaN,1988,http://books.google.nl/books?id=6HDOwAEACAAJ&d...,NaN,3.0
8,Saint Hyacinth of Poland,The story for children 10 and up of St. Hyacin...,Mary Fabyan Windeatt,http://books.google.com/books/content?id=lmLqA...,http://books.google.nl/books?id=lmLqAAAACAAJ&d...,Tan Books & Pub,2009-01-01,http://books.google.nl/books?id=lmLqAAAACAAJ&d...,Biography & Autobiography,NaN
9,Rising Sons and Daughters: Life Among Japans N...,Wardell recalls his experience as a foreign st...,Steven Wardell,NaN,http://books.google.nl/books?id=rbLZugEACAAJ&d...,Plympton PressIntl,1995,http://books.google.nl/books?id=rbLZugEACAAJ&d...,Social Science,NaN


In [ ]:
data = data.dropna(subset=["title", "description", "authors"])
data.head()

,title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,Philip Nel,http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,Biography & Autobiography,NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,David R. Ray,http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,Religion,NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,Veronica Haddon,http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,Fiction,NaN
5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,Everett Ferguson,http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,Religion,5.0
8,Saint Hyacinth of Poland,The story for children 10 and up of St. Hyacin...,Mary Fabyan Windeatt,http://books.google.com/books/content?id=lmLqA...,http://books.google.nl/books?id=lmLqAAAACAAJ&d...,Tan Books & Pub,2009-01-01,http://books.google.nl/books?id=lmLqAAAACAAJ&d...,Biography & Autobiography,NaN


In [ ]:
data.shape

(141755, 10)

from 212.403 books to 141.755 books after droping NaN values in most needed columns: title, authors, descriptions.

In [ ]:
data_selected = data[["title", "authors", "description"]]
data_selected.head()

,title,authors,description
1,Dr. Seuss: American Icon,Philip Nel,Philip Nel takes a fascinating look into the k...
2,Wonderful Worship in Smaller Churches,David R. Ray,This resource includes twelve principles in un...
3,Whispers of the Wicked Saints,Veronica Haddon,Julia Thomas finds her life spinning out of co...
5,The Church of Christ: A Biblical Ecclesiology ...,Everett Ferguson,In The Church of Christ: A Biblical Ecclesiolo...
8,Saint Hyacinth of Poland,Mary Fabyan Windeatt,The story for children 10 and up of St. Hyacin...


In [ ]:
data_selected["description_cleaned"] = data_selected["title"] + " " + data_selected["authors"] + " " + data_selected["description"]
data_selected.head()

<ipython-input-19-32584aa6b9e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected["description_cleaned"] = data_selected["title"] + " " + data_selected["authors"] + " " + data_selected["description"]


,title,authors,description,description_cleaned
1,Dr. Seuss: American Icon,Philip Nel,Philip Nel takes a fascinating look into the k...,Dr. Seuss: American Icon Philip Nel Philip Nel...
2,Wonderful Worship in Smaller Churches,David R. Ray,This resource includes twelve principles in un...,Wonderful Worship in Smaller Churches David R....
3,Whispers of the Wicked Saints,Veronica Haddon,Julia Thomas finds her life spinning out of co...,Whispers of the Wicked Saints Veronica Haddon ...
5,The Church of Christ: A Biblical Ecclesiology ...,Everett Ferguson,In The Church of Christ: A Biblical Ecclesiolo...,The Church of Christ: A Biblical Ecclesiology ...
8,Saint Hyacinth of Poland,Mary Fabyan Windeatt,The story for children 10 and up of St. Hyacin...,Saint Hyacinth of Poland Mary Fabyan Windeatt ...


In [ ]:
pd.set_option("display.max_colwidth", None)
data_selected["description_cleaned"]

,description_cleaned
1,"Dr. Seuss: American Icon Philip Nel Philip Nel takes a fascinating look into the key aspects of Seusss career - his poetry, politics, art, marketing, and place in the popular imagination."" ""Nel argues convincingly that Dr. Seuss is one of the most influential poets in America. His nonsense verse, like that of Lewis Carroll and Edward Lear, has changed language itself, giving us new words like ""nerd."" And Seusss famously loopy artistic style - what Nel terms an ""energetic cartoon surrealism"" - has been equally important, inspiring artists like filmmaker Tim Burton and illustrator Lane Smith. --from back cover"
2,"Wonderful Worship in Smaller Churches David R. Ray This resource includes twelve principles in understanding small church worship, fifteen practices for planning worship with fewer than 100 people, and suggestions for congregational study."
3,"Whispers of the Wicked Saints Veronica Haddon Julia Thomas finds her life spinning out of control after the death of her husband, Richard. Julia turns to her minister for comfort when she finds herself falling for him with a passion that is forbidden by the church. Heath Sparks is a man of God who is busy taking care of his quadriplegic wife who was seriously injured in a sever car accident. In an innocent effort to reach out to a lonely member of his church, Heath finds himself as the man and not the minister as Heath and Julia surrender their bodies to each other and face the wrath of God. Julia finds herself in over her head as she faces a deadly disease, the loss of her home and whispers about her wicked affair. Julia leaves the states offering her body as a living sacrifice in hopes of finding a cure while her heart remains thousands of miles away hoping to one day reunite with the man who holds it hostage.Whispers of the Wicked Saints is a once in a lifetime romance that is breath taking, defying all the rules of romance and bending the laws of love."
5,"The Church of Christ: A Biblical Ecclesiology for Today Everett Ferguson In The Church of Christ: A Biblical Ecclesiology for Today, respected biblical scholar Everett Ferguson presents a genuine biblical theology of the church. By systematically examining the New Testaments teaching on the existence, meaning, and purpose of the church, providing responsible coverage of the traditional topics in ecclesiology, and carefully grounding ecclesiology in the person and work of Christ, Ferguson unveils a comprehensive model of the church that is both biblically centered and relevant to a world on the verge of the twenty-first century."
8,"Saint Hyacinth of Poland Mary Fabyan Windeatt The story for children 10 and up of St. Hyacinth, the Dominican who planted the Faith in Poland, Lithuania and Russia and worked many miracles. He went to Rome, where he met St. Dominic, and was one of the first to receive at his hands the habit of the newly established Order of Friars Preachers. After his novitiate he made his religious profession, and was made superior of the little band of missionaries sent to Poland to preach. Impr. 189 pgs 16 Illus, PB"
...,...
212398,"Autodesk Inventor 10 Essentials Plus Daniel Banach, Travis Jones Autodesk Inventor 2017 Essentials Plus provides the foundation for a hands-on course that covers basic and advanced Autodesk Inventor features used to create, edit, document, and print parts and assemblies. You learn about part and assembly modeling through real-world exercises. Autodesk Inventor 2017 Essentials Plus demonstrates critical CAD concepts, from basic sketching and modeling through advanced modeling techniques, as it equips you with the skills to master this powerful professional tool. The book walks you through every component of the software, including the user interface, toolbars, dialogue boxes, sketch tools, drawing views, assembly modeling, and more. Its unique modular organization puts key information at your fingertips, while step-by-step tutorials make it an ideal res

In [ ]:
# Remove punctuation
data_selected["description_cleaned"] = data_selected["description_cleaned"].str.replace(r"[!\"#$%&()*+,-./:;<=>?@[\\\]^_``{|}~]+", " ", regex=True)

#lowercase
data_selected["description_cleaned"] = data_selected["description_cleaned"].fillna('').apply(lambda x: x.lower())

# Remove numbers
data_selected["description_cleaned"] = data_selected["description_cleaned"].apply(lambda x: "".join(ch for ch in x if not ch.isdigit()))

# Remore additional space from string
data_selected["description_cleaned"] = data_selected["description_cleaned"].apply(lambda x: re.sub(r'\s+', ' ', x).strip())


<ipython-input-21-f4f75ad38951>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected["description_cleaned"] = data_selected["description_cleaned"].str.replace(r"[!\"#$%&()*+,-./:;<=>?@[\\\]^_``{|}~]+", " ", regex=True)
<ipython-input-21-f4f75ad38951>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected["description_cleaned"] = data_selected["description_cleaned"].fillna('').apply(lambda x: x.lower())
<ipython-input-21-f4f75ad38951>:8: SettingWithCopyWarning: 
A value is trying to be set 

In [ ]:
data_selected["description_cleaned"]

,description_cleaned
1,dr seuss american icon philip nel philip nel takes a fascinating look into the key aspects of seusss career his poetry politics art marketing and place in the popular imagination nel argues convincingly that dr seuss is one of the most influential poets in america his nonsense verse like that of lewis carroll and edward lear has changed language itself giving us new words like nerd and seusss famously loopy artistic style what nel terms an energetic cartoon surrealism has been equally important inspiring artists like filmmaker tim burton and illustrator lane smith from back cover
2,wonderful worship in smaller churches david r ray this resource includes twelve principles in understanding small church worship fifteen practices for planning worship with fewer than people and suggestions for congregational study
3,whispers of the wicked saints veronica haddon julia thomas finds her life spinning out of control after the death of her husband richard julia turns to her minister for comfort when she finds herself falling for him with a passion that is forbidden by the church heath sparks is a man of god who is busy taking care of his quadriplegic wife who was seriously injured in a sever car accident in an innocent effort to reach out to a lonely member of his church heath finds himself as the man and not the minister as heath and julia surrender their bodies to each other and face the wrath of god julia finds herself in over her head as she faces a deadly disease the loss of her home and whispers about her wicked affair julia leaves the states offering her body as a living sacrifice in hopes of finding a cure while her heart remains thousands of miles away hoping to one day reunite with the man who holds it hostage whispers of the wicked saints is a once in a lifetime romance that is breath taking defying all the rules of romance and bending the laws of love
5,the church of christ a biblical ecclesiology for today everett ferguson in the church of christ a biblical ecclesiology for today respected biblical scholar everett ferguson presents a genuine biblical theology of the church by systematically examining the new testaments teaching on the existence meaning and purpose of the church providing responsible coverage of the traditional topics in ecclesiology and carefully grounding ecclesiology in the person and work of christ ferguson unveils a comprehensive model of the church that is both biblically centered and relevant to a world on the verge of the twenty first century
8,saint hyacinth of poland mary fabyan windeatt the story for children and up of st hyacinth the dominican who planted the faith in poland lithuania and russia and worked many miracles he went to rome where he met st dominic and was one of the first to receive at his hands the habit of the newly established order of friars preachers after his novitiate he made his religious profession and was made superior of the little band of missionaries sent to poland to preach impr pgs illus pb
...,...
212398,autodesk inventor essentials plus daniel banach travis jones autodesk inventor essentials plus provides the foundation for a hands on course that covers basic and advanced autodesk inventor features used to create edit document and print parts and assemblies you learn about part and assembly modeling through real world exercises autodesk inventor essentials plus demonstrates critical cad concepts from basic sketching and modeling through advanced modeling techniques as it equips you with the skills to master this powerful professional tool the book walks you through every component of the software including the user interface toolbars dialogue boxes sketch tools drawing views assembly modeling and more its unique modular organization puts key information at your fingertips while step by step tutorials make it an ideal resource for self learning packed with vivid illustrations and practical exercises that emphasize modern day applications autodesk

In [ ]:
# Loading english NLP model
nlp = en_core_web_sm.load()

In [ ]:
# Remove stop-words and lemmatize tokens
tokenized_doc = data_selected["description_cleaned"].apply(lambda x: [token.lemma_ for token in nlp(x) if token.text not in STOP_WORDS])

In [ ]:
tokenized_doc = data_selected["description_cleaned"].apply(lambda x: [token.lemma_ for token in nlp(x) if token.text not in STOP_WORDS])

In [ ]:
tokenized_doc.head()

,description_cleaned
1,"[dr, seuss, american, icon, philip, nel, phili..."
2,"[wonderful, worship, small, church, david, r, ..."
3,"[whisper, wicked, saints, veronica, haddon, ju..."
5,"[church, christ, biblical, ecclesiology, today..."
8,"[saint, hyacinth, poland, mary, fabyan, windea..."


In [ ]:
data_selected["token"] = tokenized_doc.apply(lambda x: ' '.join(x))
data_selected["token"]

<ipython-input-35-a110a9cc4989>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected["token"] = tokenized_doc.apply(lambda x: ' '.join(x))


,token
1,dr seuss american icon philip nel philip nel t...
2,wonderful worship small church david r ray res...
3,whisper wicked saints veronica haddon julia th...
5,church christ biblical ecclesiology today ever...
8,saint hyacinth poland mary fabyan windeatt sto...
...,...
212398,autodesk inventor essential plus daniel banach...
212399,orphan ellis island time travel adventure elvi...
212401,mamaw wild wild cabbage mamaw useful beautiful...
212402,autograph man zadie smith alex li tandem sell ...


In [ ]:
pd.reset_option("display.max_colwidth")

1. Visualize the Frequency Distribution of Tokens
A good first step is to visualize the token frequency distribution to understand how your tokens are distributed. If you plot the distribution, you can better decide what "high-frequency" means in the context of your data.



```
all_tokens = [token for tokens in tokenized_doc for token in tokens]

# Plot the distribution of token frequencies
token_counts = pd.Series(all_tokens).value_counts()
plt.figure(figsize=(10,6))
token_counts.plot(kind='hist', bins=50)
plt.title('Token Frequency Distribution')
plt.xlabel('Frequency')
plt.ylabel('Number of Tokens')
plt.show()

```



## How to define the number of token value count to remove (remove most common token used)

Option 1: Instead of using an arbitrary number like 20 or 10, you could remove tokens that fall within the top X percentile of token frequencies. For example, you could choose to remove the top 1% or 5% of the most frequent tokens.


code:

```
all_tokens = [token for tokens in tokenized_doc for token in tokens]
token_counts = pd.Series(all_tokens).value_counts()
# Choose the percentile threshold (e.g., 1% most frequent tokens)
percentile = 0.01  # top 1% most frequent tokens
top_percentile_threshold = token_counts.quantile(1 - percentile)

# Get tokens that exceed the percentile threshold
tokens_to_remove = token_counts[token_counts > top_percentile_threshold]

tokens_to_remove

```


Option 2: Test Different Values for Threshold and Evaluate
You can start with a range of values (10, 20, 50, 100, etc.), and evaluate which one produces the best results for your analysis. Here's how you can experiment:


```
# Try different threshold values and observe results
thresholds = [10, 20, 50, 100]
for threshold in thresholds:
    tokens_to_remove = token_counts[token_counts > threshold].index
    # Apply filtering
    tokenized_doc_filtered = tokenized_doc.apply(lambda tokens: [token for token in tokens if token not in tokens_to_remove])
    
    # Evaluate the effect (e.g., print the number of tokens removed)
    removed_tokens_count = sum([len(tokens_to_remove.intersection(set(tokens))) for tokens in tokenized_doc_filtered])
    print(f"Threshold: {threshold}, Tokens removed: {removed_tokens_count}")

```

Once the evaluation done:

```
all_tokens = [token for tokens in tokenized_doc for token in tokens]
token_counts = pd.Series(all_tokens).value_counts()
tokens_to_remove = token_counts[token_counts > 10]
tokenized_doc = tokenized_doc.apply(lambda tokens: [token for token in tokens if token not in tokens_to_remove])

data_selected["token"] = tokenized_doc.apply(lambda x: ' '.join(x))
data_selected["token"]

```





In [ ]:
all_tokens = [token for tokens in tokenized_doc for token in tokens]
token_counts = pd.Series(all_tokens).value_counts()
# Choose the percentile threshold (e.g., 1% most frequent tokens)
percentile = 0.01  # top 1% most frequent tokens
top_percentile_threshold = token_counts.quantile(1 - percentile)

# Get tokens that exceed the percentile threshold
tokens_to_remove = token_counts[token_counts > top_percentile_threshold]

In [ ]:
tokens_to_remove

,count
book,82015
new,53569
life,49768
world,38084
include,36337
...,...
fighter,760
joyce,760
musician,760
mount,760


In [ ]:
tokenized_doc_updated = tokenized_doc.apply(lambda tokens: [token for token in tokens if token not in tokens_to_remove])

data_selected["token"] = tokenized_doc_updated.apply(lambda x: ' '.join(x))
data_selected["token"]

<ipython-input-41-87ddbe829266>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected["token"] = tokenized_doc_updated.apply(lambda x: ' '.join(x))


,token
1,seuss icon nel nel seusss nel convincingly seu...
2,few congregational
3,whisper wicked saints veronica haddon julia sp...
5,ecclesiology everett ferguson ecclesiology res...
8,hyacinth poland fabyan windeatt hyacinth domin...
...,...
212398,autodesk inventor banach travis autodesk inven...
212399,orphan ellis elvira woodruff ellis dominick av...
212401,mamaw cabbage mamaw thoughtful shell pretty bl...
212402,autograph zadie li tandem autographs occasiona...


In [ ]:
pd.set_option("display.max_colwidth", None)
data_selected.head()

,title,authors,description,description_cleaned,token
1,Dr. Seuss: American Icon,Philip Nel,"Philip Nel takes a fascinating look into the key aspects of Seusss career - his poetry, politics, art, marketing, and place in the popular imagination."" ""Nel argues convincingly that Dr. Seuss is one of the most influential poets in America. His nonsense verse, like that of Lewis Carroll and Edward Lear, has changed language itself, giving us new words like ""nerd."" And Seusss famously loopy artistic style - what Nel terms an ""energetic cartoon surrealism"" - has been equally important, inspiring artists like filmmaker Tim Burton and illustrator Lane Smith. --from back cover",dr seuss american icon philip nel philip nel takes a fascinating look into the key aspects of seusss career his poetry politics art marketing and place in the popular imagination nel argues convincingly that dr seuss is one of the most influential poets in america his nonsense verse like that of lewis carroll and edward lear has changed language itself giving us new words like nerd and seusss famously loopy artistic style what nel terms an energetic cartoon surrealism has been equally important inspiring artists like filmmaker tim burton and illustrator lane smith from back cover,seuss icon nel nel seusss nel convincingly seuss nonsense carroll lear nerd seusss famously loopy nel energetic cartoon surrealism equally filmmaker burton illustrator lane
2,Wonderful Worship in Smaller Churches,David R. Ray,"This resource includes twelve principles in understanding small church worship, fifteen practices for planning worship with fewer than 100 people, and suggestions for congregational study.",wonderful worship in smaller churches david r ray this resource includes twelve principles in understanding small church worship fifteen practices for planning worship with fewer than people and suggestions for congregational study,few congregational
3,Whispers of the Wicked Saints,Veronica Haddon,"Julia Thomas finds her life spinning out of control after the death of her husband, Richard. Julia turns to her minister for comfort when she finds herself falling for him with a passion that is forbidden by the church. Heath Sparks is a man of God who is busy taking care of his quadriplegic wife who was seriously injured in a sever car accident. In an innocent effort to reach out to a lonely member of his church, Heath finds himself as the man and not the minister as Heath and Julia surrender their bodies to each other and face the wrath of God. Julia finds herself in over her head as she faces a deadly disease, the loss of her home and whispers about her wicked affair. Julia leaves the states offering her body as a living sacrifice in hopes of finding a cure while her heart remains thousands of miles away hoping to one day reunite with the man who holds it hostage.Whispers of the Wicked Saints is a once in a lifetime romance that is breath taking, defying all the rules of romance and bending the laws of love.",whispers of the wicked saints veronica haddon julia thomas finds her life spinning out of control after the death of her husband richard julia turns to her minister for comfort when she finds herself falling for him with a passion that is forbidden by the church heath sparks is a man of god who is busy taking care of his quadriplegic wife who was seriously injured in a sever car accident in an innocent effort to reach out to a lonely member of his church heath finds himself as the man and not the minister as heath and julia surrender their bodies to each other and face the wrath of god julia finds herself in over her head as she faces a deadly disease the loss of her home and whispers about her wicked affair julia leaves the states offering her body as a living sacrifice in hopes of finding a cure while her heart remains thousands of miles away hoping to one day reunite with the man who holds it hostage whispers of the wicked saints is a once in a lifetime romance that is br

In [ ]:
#data_selected.to_csv("/content/sample_data/data_selected_token_1percent.csv", index=False)

NameError: name 'data_selected' is not defined

In [ ]:
#from google.colab import files
#files.download('/content/sample_data/data_selected_token_1percent.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_selected = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/book_recommendation_project/data_selected.csv", index_col=0)
data_selected.head()

,authors,description,description_cleaned,token
title,,,,
Dr. Seuss: American Icon,Philip Nel,Philip Nel takes a fascinating look into the k...,dr seuss american icon philip nel philip nel t...,dr seuss american icon philip nel philip nel t...
Wonderful Worship in Smaller Churches,David R. Ray,This resource includes twelve principles in un...,wonderful worship in smaller churches david r ...,wonderful worship small church david r ray res...
Whispers of the Wicked Saints,Veronica Haddon,Julia Thomas finds her life spinning out of co...,whispers of the wicked saints veronica haddon ...,whisper wicked saints veronica haddon julia th...
The Church of Christ: A Biblical Ecclesiology for Today,Everett Ferguson,In The Church of Christ: A Biblical Ecclesiolo...,the church of christ a biblical ecclesiology f...,church christ biblical ecclesiology today ever...
Saint Hyacinth of Poland,Mary Fabyan Windeatt,The story for children 10 and up of St. Hyacin...,saint hyacinth of poland mary fabyan windeatt ...,saint hyacinth poland mary fabyan windeatt sto...


In [ ]:
#data_selected = pd.read_csv("data_selected.csv", index_col=0)
#data_selected.head()

In [ ]:
# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(data_selected["token"])

In [ ]:
# Get tokens for all documents
vectorizer.vocabulary_

{'dr': 52554,
 'seuss': 170421,
 'american': 5809,
 'icon': 88575,
 'philip': 145006,
 'nel': 130736,
 'fascinating': 63676,
 'look': 111873,
 'key': 100639,
 'aspect': 11095,
 'seusss': 170425,
 'career': 29175,
 'poetry': 147519,
 'politic': 147790,
 'art': 10375,
 'marketing': 117152,
 'place': 146585,
 'popular': 148381,
 'imagination': 89459,
 'argue': 9727,
 'convincingly': 39205,
 'influential': 91209,
 'poet': 147498,
 'america': 5801,
 'nonsense': 133228,
 'verse': 201964,
 'like': 109945,
 'lewis': 109091,
 'carroll': 29609,
 'edward': 55733,
 'lear': 107501,
 'change': 31696,
 'language': 106280,
 'new': 131475,
 'word': 209777,
 'nerd': 130950,
 'famously': 63255,
 'loopy': 111920,
 'artistic': 10511,
 'style': 181894,
 'term': 187796,
 'energetic': 58187,
 'cartoon': 29706,
 'surrealism': 183626,
 'equally': 59275,
 'important': 89925,
 'inspire': 92107,
 'artist': 10507,
 'filmmaker': 65419,
 'tim': 190607,
 'burton': 26719,
 'illustrator': 89349,
 'lane': 106193,
 'smith

In [ ]:
#vocab_size = 100

#data_selected_f = pd.DataFrame(X.transpose().todense())
#data_selected_f['mean'] = data_selected_f.mean(axis=1)
#data_selected_f = data_selected_f.sort_values(by='mean', ascending=False)#[:vocab_size]
#data_selected_f['token_id'] = data_selected_f.index

#data_selected_tokens = pd.DataFrame({'token':vectorizer.get_feature_names_out()})
#data_selected_tokens['token_id'] = data_selected_tokens.index

#data_selected_f = data_selected_f.merge(data_selected_tokens, on='token_id')

In [ ]:
#data_selected_f.head(20)

In [ ]:
#Find best params for DSCAN
scores_data_selected = pd.DataFrame(columns = ["eps", "min_samples", "clusters", "noise_points"])

for i in np.arange(0.1, 1, 0.01):
    for samples in np.arange(3, 20, 1):
        db = DBSCAN(eps=i, min_samples=samples, metric="cosine")
        db.fit(X)

        # Check the number of clusters (excluding noise)
        n_clusters_ = len(np.unique(db.labels_)) - (1 if -1 in db.labels_ else 0)

        # Save in a df results where there're between 10/20 clusters and not too many noise
        if n_clusters_ >= 10 and n_clusters_ < 20 and list(db.labels_).count(-1) <= 100 :
            new_row = pd.DataFrame({"eps": [i], "min_samples": [samples], "clusters": n_clusters_,"noise_points": [list(db.labels_).count(-1)]})
            scores_df = pd.concat([scores_data_selected, new_row], ignore_index=True)

        else:
            pass

In [ ]:
scores_df.sort_values(by = "noise_points", ascending = True)

In [ ]:
#replace params by best params finds for DSCAN above
db = DBSCAN(eps=0.72, min_samples=4, metric="cosine", algorithm="auto")
db.fit(X)

In [ ]:
np.unique(db.labels_)

In [ ]:
data_selected["cluster_DBSCAN"] = db.labels_
data_selected.head()

In [ ]:
mask0 = data_selected["cluster_DBSCAN"] == 0
data_selected[mask0]

In [ ]:
mask1 = data_selected["cluster_DBSCAN"] == 1
data_selected[mask1]

In [ ]:
wd = wordcloud.WordCloud()

for i in range(len(np.unique(db.labels_))-1):
    corpus=[]
    for x in data_selected["token"][data_selected["cluster_DBSCAN"]==i]:
        corpus.append(x)

    to_cloud = "" "".join(corpus)
    cloud = wd.generate(to_cloud)
    plt.title(f"Cluster {i}")
    plt.imshow(cloud)
    plt.show()

## Recommendation system:

In [ ]:
def find_similar_items(title):
    item_cluster = data_selected[data_selected['title']==title]["cluster_DBSCAN"].iloc[0]
    df_cluster_fonction = data_selected[data_selected["cluster_DBSCAN"]==item_cluster]
    return df_cluster_fonction.sample(n=10)[["title", "author", "description", "cluster_DBSCAN"]]

In [ ]:
condition = True
while condition == True:
    id=input("Select an item id (from 1 to 500 included):")

    if str(id).isnumeric()==False:
        condition=True

    elif int(id) <1 or int(id) >500:
        condition=True

    else:
        display(find_similar_items(id))
        condition=False
